# Explainable Persian Question Answering with **BERT**
## by **[Abolfazl Mohajeri](https://abolfazlmohajeri.ir)**

In [6]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import numpy as np
import pandas as pd
import torch

!pip install captum
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
model_path = 'SajjadAyoubi/bert-base-fa-qa'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [8]:
# Make predictions method
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
    return output.start_logits, output.end_logits

# Custom forward function that will allow us to access the start and end positions of our prediction using position input argument
def custom_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
    pred = pred[position]
    return pred.max(1).values

# Construct input_ids, ref_input_ids and sep_id
def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # Construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]
    # Construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

# Construct token_type_ids include 0, 1
def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    return token_type_ids

# Construct position id for each input token
def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

# Summarize attributions for each word token in the sequence
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [9]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [10]:
text = "کال آو دیوتی یا ندای وظیفه (به انگلیسی: Call of Duty) مجموعه ای از بازی های ویدئویی در گونهٔ تیراندازی اول شخص است که توسط کمپانی اکتیویژن منتشر شده است. این کمپانی ابتدا در سال ۲۰۰۳ بر روی بازی هایی که داستانشان در جنگ جهانی دوم جریان داشتند متمرکز بود."
question = "شرکت سازنده کال آو دیوتی چیه؟"

# ref_input_ids use as IG baselines
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)
print("input_ids is:", input_ids)
print("ref_input_ids is: (used for baseline)", ref_input_ids)
print("sep index is:", sep_id)
token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
print("token_type_ids is:", token_type_ids)
position_ids = construct_input_ref_pos_id_pair(input_ids)
print("position_ids is:", position_ids)
attention_mask = construct_attention_mask(input_ids)
print("attention_mask is:", attention_mask)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

ground_truth = 'اکتیویژن'
ground_truth_tokens = tokenizer.encode(ground_truth, add_special_tokens=False)
ground_truth_end_ind = indices.index(ground_truth_tokens[-1])
ground_truth_start_ind = ground_truth_end_ind - len(ground_truth_tokens) + 1

input_ids is: tensor([[    2,  3052,  7289,  3709,  2866, 74398, 21628,  1350,     4,  3709,
          2866, 74398,  2880, 17563,  5717,  1006,  2789,  4522,  1014, 20984,
          6840, 29027,  1007,  3855,  2938,  2791,  3057,  6343,  9786,  2786,
          4600, 10855,  3067,  3504,  2806,  2800,  3158,  5944, 33238,  3692,
          2871,  2806,  1012,  2802,  5944,  4309,  2786,  2844,  8043,  2801,
          3040,  3057,  4016,  2800, 29670,  2783,  2786,  3324,  3553,  3358,
          3877,  4159,  7425,  2834,  1012,     4]], device='cuda:0')
ref_input_ids is: (used for baseline) tensor([[2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]],
       device='cuda:0')
sep index is: 7
token_type_ids is: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
start_scores, end_scores = predict(input_ids, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
print('Question: ', question)
print('Predicted Answer: ', ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

Question:  شرکت سازنده کال آو دیوتی چیه؟
Predicted Answer:  اکتیویژن


In [12]:
lig = LayerIntegratedGradients(custom_forward_func, model.bert.embeddings)

attributions_start, delta_start = lig.attribute(inputs=input_ids, baselines=ref_input_ids, additional_forward_args=(token_type_ids, position_ids, attention_mask, 0), return_convergence_delta=True)
attributions_end, delta_end = lig.attribute(inputs=input_ids, baselines=ref_input_ids, additional_forward_args=(token_type_ids, position_ids, attention_mask, 1), return_convergence_delta=True)
attributions_start_sum = summarize_attributions(attributions_start)
attributions_end_sum = summarize_attributions(attributions_end)

start_position_vis = viz.VisualizationDataRecord(
                        attributions_start_sum,
                        torch.max(torch.softmax(start_scores[0], dim=0)),
                        torch.argmax(start_scores),
                        torch.argmax(start_scores),
                        str(ground_truth_start_ind),
                        attributions_start_sum.sum(),       
                        all_tokens,
                        delta_start)
end_position_vis = viz.VisualizationDataRecord(
                        attributions_end_sum,
                        torch.max(torch.softmax(end_scores[0], dim=0)),
                        torch.argmax(end_scores),
                        torch.argmax(end_scores),
                        str(ground_truth_end_ind),
                        attributions_end_sum.sum(),       
                        all_tokens,
                        delta_end)

print('Visualizations For Start Position')
viz.visualize_text([start_position_vis])
print('Visualizations For End Position')
viz.visualize_text([end_position_vis])

Visualizations For Start Position


Visualizations For End Position
